In [1]:
import numpy as np
import os
from embeddings import SentenceEmbeddingDataset, WordEmbeddingManager, create_embedding_dataloader
import utils
import torch

/Users/johnhenryrudden/anaconda3/envs/shaky_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/johnhenryrudden/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
TRAIN_PATH = 'data/raw_train.txt'
tokenized_sentences = utils.process_training_data(TRAIN_PATH)

In [3]:
WORD2VEC_MODEL_PATH = 'data/word2vec.model'
word2vec_manager = WordEmbeddingManager(WORD2VEC_MODEL_PATH)

Model loaded successfully from data/word2vec.model


In [4]:
dataloader = create_embedding_dataloader(tokenized_sentences, word2vec_manager, seq_length=10, batch_size=4)

In [6]:
next(iter(dataloader))

tensor([[[ 2.4565e-02,  5.1748e-01, -5.9399e-01,  ..., -4.1718e-01,
           7.6922e-01,  5.4657e-01],
         [-1.3729e-01,  5.1425e-01, -4.8553e-01,  ..., -4.0286e-01,
           5.1330e-01,  2.8110e-01],
         [-1.4272e-01,  1.4315e+00, -1.7884e-02,  ..., -6.7034e-01,
           4.4811e-01,  1.0103e+00],
         ...,
         [ 9.4943e-04,  5.0629e-02, -4.4952e-02,  ..., -3.9130e-02,
           6.1908e-02,  1.8515e-02],
         [-1.7381e-02,  8.8856e-02, -1.1984e-01,  ..., -9.7379e-02,
           1.5058e-01,  1.1743e-01],
         [-4.2222e-01,  6.5892e-01, -3.0751e-01,  ..., -3.9470e-01,
           8.3220e-01,  1.0330e+00]],

        [[-6.3585e-02,  6.9724e-01, -6.9868e-01,  ..., -5.0057e-01,
           7.8134e-01,  6.3834e-01],
         [-1.0421e-01,  6.8249e-01, -8.9117e-01,  ..., -7.5829e-01,
           7.2251e-01,  2.4830e-01],
         [ 2.9479e-01,  5.2084e-01, -3.0711e-01,  ..., -4.7313e-01,
           6.5089e-01,  1.2639e+00],
         ...,
         [-4.0002e-02,  2